<a href="https://colab.research.google.com/github/yusufkhan004/Machine-learning-tasks/blob/master/text%20generation/text_generation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#load data
#loading data and opening our input data in th e form of text file
#project Gutenburg/berg is where the data can be found(just Google it)
file = open('thinkbayes.txt').read()

In [0]:
#import independencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras import Sequential  #For building the Neural Network layer by layer
from tensorflow.keras.layers import Dense, Dropout,LSTM  # dense -->To randomly initialize the weights to small numbers close to 0(But not 0)
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
#tokenization = process of breaking down a stream of text into a words,phrases or symbols or other such meaningful elements which are called tokens,
#list of tokens become imputs for futher process suh as parsing or text mining. simply ,tokenization is a method to simplify content prior to the next step of processing it
#before a tokenization of data we'll make all the words in lowercase in oreder to standardize it
#standardization
def tokenize_words(input):
  input = input.lower()
  tokenizer = RegexpTokenizer(r'\w+') #instantiating a tokenizer
  tokens = tokenizer.tokenize(input)  #
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens) #common words like a,the,an etc thay dont provide any meaning to the data we are going to use so we have to filter them using lambda method
  return "".join(filtered)
  
processed_inputs = tokenize_words(file)  # we r going to preprocess the data

In [0]:
 #chars to numbers
 #convert character in our input to numbers
 #we'll sort the list of the set of all characters that appears os out i/p text and then use the enumerate function to get numbers that reprsent the character
 #we'll then create a dictionary that stores the keys and valus , or the characters and the numbers that reresent them
 chars = sorted(list(set(processed_inputs)))
 chars_to_nums = dict((c,i) for i, c in enumerate(chars))

In [0]:
#check if words to chars or chars to num(?!) has worked ?
#just so we get an idea of whether our process of converting words to characters has worked
#we print the length of the variables
input_len = len(processed_inputs)
vocab_len = len(chars)
print('total numbers of characters:', input_len)
print('total vocab:',vocab_len)

total numbers of characters: 173092
total vocab: 46


In [0]:
#sequence length
#we're defining how long an individual sequence here
#an individual sequence is a complete mapping of inputcharacteristtics as integers
seq_length = 100
x_data = []
y_data = []


In [0]:
#loop through the sequence
#here we're going through entire list of i/ps and converting char into nuberswith a for loop
#tjis wiil create a bunch of sequences where each sequence start with the next characters is the i/p data
#beginning with the first character
for i in range(0,input_len - seq_length, 1):    
  in_seq =processed_inputs[i:i + seq_length]  #define i/p and o/p sequence. input is the current character + desired sequence length
  out_seq = processed_inputs[i + seq_length]  #out sequence is the iniial charcter + total sequence length
  x_data.append([chars_to_nums[char] for char in in_seq])  #converting the lists of char to int based on previous alue and appending the values to our lists
  y_data.append(chars_to_nums[out_seq])

n_patterns = len(x_data)  #check to see how many total input sequence we have
print("Total Patterns:",n_patterns)

Total Patterns: 172992


In [0]:
#convert input sequence to np array that our network can use
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)


In [0]:
# one-hot encoding our label data
y = np_utils.to_categorical(y_data)


In [0]:
#creating the model
#creating a sequential model
#dropout is used to prevent ovrfitting
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
#compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [0]:
#saving weights
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose =1, save_best_only=True, mode='min')
desired_callbacks= [checkpoint]

In [14]:
#fit model and let it train
model.fit(X,y, epochs = 4, batch_size=254, callbacks=desired_callbacks)

Epoch 1/4
682/682 [==============================] - ETA: 0s - loss: 3.1259
Epoch 00001: loss improved from inf to 3.12592, saving model to model_weights_saved.hdf5
682/682 [==============================] - 2610s 4s/step - loss: 3.1259
Epoch 2/4
682/682 [==============================] - ETA: 0s - loss: 3.0778
Epoch 00002: loss improved from 3.12592 to 3.07784, saving model to model_weights_saved.hdf5
682/682 [==============================] - 2628s 4s/step - loss: 3.0778
Epoch 3/4
682/682 [==============================] - ETA: 0s - loss: 2.9708
Epoch 00003: loss improved from 3.07784 to 2.97085, saving model to model_weights_saved.hdf5
682/682 [==============================] - 2585s 4s/step - loss: 2.9708
Epoch 4/4
682/682 [==============================] - ETA: 0s - loss: 2.8685
Epoch 00004: loss improved from 2.97085 to 2.86846, saving model to model_weights_saved.hdf5
682/682 [==============================] - 2557s 4s/step - loss: 2.8685


In [0]:
#recompile module with thee saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [0]:
#output of the model back into characters
num_to_char = dict((i,e) for i,e in enumerate(chars))

In [17]:
#random need to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seeds:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")


Random Seeds:
" ldifferentialpositivebruinswinnegativecanuckswin0tiep_windiffprobgreater0p_lossdiffprobless0p_tiedif "


In [18]:
#generate the text
for i in range (1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in =[num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
  

eoeeeoeeeateseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseeseseesesee